In [1]:
import os
import sys
import subprocess
import random
import warnings
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from tqdm.notebook import tqdm
import itertools

import torch
from qucumber.nn_states import DensityMatrix
from qucumber.nn_states import ComplexWaveFunction
from qucumber.callbacks import MetricEvaluator
import qucumber.utils.unitaries as unitaries
import qucumber.utils.training_statistics as ts
import qucumber.utils.cplx as cplx
import qucumber.utils.data as data
from qucumber.observables import ObservableBase, to_pm1
from qucumber.observables.pauli import flip_spin
import qucumber

from qulacs.gate import Pauli

import optuna

with open('./params_setting.yaml', 'r') as yml:
    params = yaml.safe_load(yml)
    
# quantum circuit parameter
n_qubit = params["circuit_info"]["n_qubit"]
each_n_shot = params["circuit_info"]["each_n_shot"]
state_name = params["circuit_info"]["state_name"]
error_model = params["circuit_info"]["error_model"]
error_rate = params["circuit_info"]["error_rate"]
# RBM architecture parameter
n_visible_unit = params["architecture_info"]["n_visible_unit"]
n_hidden_unit = params["architecture_info"]["n_hidden_unit"] 
n_aux_unit = params["architecture_info"]["n_aux_unit"]
# train parameter
lr = params["train_info"]["lr"]
pbs = params["train_info"]["positive_batch_size"]
nbs = params["train_info"]["negative_batch_size"]
n_gibbs_step = params["train_info"]["n_gibbs_step"]
period = 1
epoch = params["train_info"]["n_epoch"]
lr_drop_epoch = params["train_info"]["lr_drop_epoch"]
lr_drop_factor = params["train_info"]["lr_drop_factor"]
seed = params["train_info"]["seed"]
# sampling parameter
n_sampling = params["sampling_info"]["n_sample"]
n_copy = params["sampling_info"]["n_copy"]
# data path info
train_data_path = f"./data/{error_model}/error_prob_{100*error_rate}%/num_of_data_{each_n_shot}/"
ideal_state_path = f"./target_state/"

# settings
## warnings
warnings.simplefilter('ignore')

## seaborn layout
sns.set()
sns.set_style("white")

## seed
def seed_settings(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    qucumber.set_random_seed(seed, cpu=True, gpu=False)

seed_settings(seed=seed)

In [2]:
meas_pattern_path = train_data_path + "/measurement_pattern.txt"
meas_label_path = train_data_path + "/measurement_label.txt"
meas_result_path = train_data_path + "/measurement_result.txt"
ideal_rho_re_path = ideal_state_path + "/rho_real.txt"
ideal_rho_im_path = ideal_state_path + "/rho_imag.txt"
meas_result, ideal_rho, meas_label, meas_pattern = data.load_data_DM(meas_result_path,
                                                                     ideal_rho_re_path,
                                                                     ideal_rho_im_path,
                                                                     meas_label_path,
                                                                     meas_pattern_path)

In [3]:
nn_state_dm = DensityMatrix(
    num_visible = n_visible_unit, 
    num_hidden = n_hidden_unit, 
    num_aux = n_aux_unit, 
    unitary_dict = unitaries.create_dict(),
    gpu = True
)

In [4]:
def create_callback_dm(nn_state):
    metric_dict = {
        "Fidelity": ts.fidelity,
        "KL_Divergence": ts.KL,
        #"Observable_XXX_ev": observable_XXX_ev,
        #"Observable_XZZ_ev": observable_XZZ_ev,
    }

    space = nn_state.generate_hilbert_space()
    callbacks = [
        MetricEvaluator(
            period,
            metric_dict,
            target = ideal_rho,
            bases = meas_pattern,
            verbose = True,
            space = space,
        )
    ]
    return callbacks

callbacks = create_callback_dm(nn_state_dm)

In [5]:
def objective(trial):
    # load dataset
    meas_pattern_path = train_data_path + "/measurement_pattern.txt"
    meas_label_path = train_data_path + "/measurement_label.txt"
    meas_result_path = train_data_path + "/measurement_result.txt"
    ideal_rho_re_path = ideal_state_path + "/rho_real.txt"
    ideal_rho_im_path = ideal_state_path + "/rho_imag.txt"
    meas_result, ideal_rho, meas_label, meas_pattern = data.load_data_DM(meas_result_path, ideal_rho_re_path, ideal_rho_im_path, meas_label_path, meas_pattern_path)
    # search params
    lr = trial.suggest_float("lr", 5, 20, log=True)
    #n_gibbs_step = trial.suggest_int("k", 10, 5000, log=True)
    pbs = trial.suggest_categorical("pbs", [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000])
    nbs = trial.suggest_categorical("nbs", [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000])
    
    nn_state_dm.fit(data = meas_result,
                            input_bases = meas_label,
                            epochs = epoch,
                            pos_batch_size = pbs,
                            neg_batch_size = nbs,
                            lr = lr,
                            k = n_gibbs_step,
                            bases = meas_pattern,
                            callbacks = callbacks,
                            time = True,
                            optimizer = torch.optim.Adadelta,
                            schexduler = torch.optim.lr_scheduler.StepLR,
                            scheduler_args = {"step_size": lr_drop_epoch, "gamma": lr_drop_factor},
                            )
        
    loss = callbacks[0]["KL_Divergence"][-1]
    trial.report(loss, epoch)
    if trial.should_prune():
        raise optuna.TrialPruned()
        
    return callbacks[0]["KL_Divergence"][-1]

In [6]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=1000)

[I 2023-08-25 03:40:01,556] A new study created in memory with name: no-name-5940753c-5a61-4661-97c1-9a5f1e69ac4e


A new study created in memory with name: no-name-5940753c-5a61-4661-97c1-9a5f1e69ac4e
Epoch: 1	Fidelity = 0.318301	KL_Divergence = 0.541188
Epoch: 2	Fidelity = 0.299754	KL_Divergence = 0.519594
Epoch: 3	Fidelity = 0.302634	KL_Divergence = 0.497503
Epoch: 4	Fidelity = 0.313960	KL_Divergence = 0.473510
Epoch: 5	Fidelity = 0.305669	KL_Divergence = 0.451718
Epoch: 6	Fidelity = 0.292351	KL_Divergence = 0.434447
Epoch: 7	Fidelity = 0.293377	KL_Divergence = 0.416233
Epoch: 8	Fidelity = 0.294965	KL_Divergence = 0.399730
Epoch: 9	Fidelity = 0.302871	KL_Divergence = 0.384644
Epoch: 10	Fidelity = 0.310355	KL_Divergence = 0.366455
Epoch: 11	Fidelity = 0.323802	KL_Divergence = 0.348863
Epoch: 12	Fidelity = 0.349900	KL_Divergence = 0.326094
Epoch: 13	Fidelity = 0.371227	KL_Divergence = 0.304170
Epoch: 14	Fidelity = 0.397929	KL_Divergence = 0.279986
Epoch: 15	Fidelity = 0.432628	KL_Divergence = 0.251524
Epoch: 16	Fidelity = 0.456764	KL_Divergence = 0.233157
Epoch: 17	Fidelity = 0.479364	KL_Divergence

[I 2023-08-25 03:40:48,606] Trial 0 finished with value: 0.10645534226558982 and parameters: {'lr': 9.109756405618683, 'pbs': 3000, 'nbs': 6000}. Best is trial 0 with value: 0.10645534226558982.


Fidelity = 0.702269	KL_Divergence = 0.106455
Total time elapsed during training: 46.894 s
Trial 0 finished with value: 0.10645534226558982 and parameters: {'lr': 9.109756405618683, 'pbs': 3000, 'nbs': 6000}. Best is trial 0 with value: 0.10645534226558982.
Epoch: 1	Fidelity = 0.707015	KL_Divergence = 0.082110
Epoch: 2	Fidelity = 0.706724	KL_Divergence = 0.084165
Epoch: 3	Fidelity = 0.705035	KL_Divergence = 0.085750
Epoch: 4	Fidelity = 0.714354	KL_Divergence = 0.085046
Epoch: 5	Fidelity = 0.728563	KL_Divergence = 0.073979
Epoch: 6	Fidelity = 0.732551	KL_Divergence = 0.076954
Epoch: 7	Fidelity = 0.729026	KL_Divergence = 0.076494
Epoch: 8	Fidelity = 0.739942	KL_Divergence = 0.070459
Epoch: 9	Fidelity = 0.732223	KL_Divergence = 0.086407
Epoch: 10	Fidelity = 0.737509	KL_Divergence = 0.072902
Epoch: 11	Fidelity = 0.747676	KL_Divergence = 0.068776
Epoch: 12	Fidelity = 0.728746	KL_Divergence = 0.085402
Epoch: 13	Fidelity = 0.741638	KL_Divergence = 0.084104
Epoch: 14	Fidelity = 0.749864	KL_Dive

[I 2023-08-25 03:41:27,362] Trial 1 finished with value: 0.05588400346209574 and parameters: {'lr': 13.576484164193129, 'pbs': 8000, 'nbs': 1000}. Best is trial 1 with value: 0.05588400346209574.


...

[W 2023-08-25 12:35:21,710] Trial 687 failed with parameters: {'lr': 19.31220073304095, 'pbs': 9000, 'nbs': 10000} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_39495/1699494747.py", line 15, in objective
    nn_state_dm.fit(data = meas_result,
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/density_matrix.py", line 354, in fit
    super().fit(
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/neural_state.py", line 614, in fit
    all_grads = self.compute_batch_gradients(k, *batch)
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/neural_state.py", line 439, in compute_batch_gradients
    grad = self.positive_phase_gradients(samples_batch, bases_batch=bases_batch)
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/neural_state

Trial 687 failed with parameters: {'lr': 19.31220073304095, 'pbs': 9000, 'nbs': 10000} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_39495/1699494747.py", line 15, in objective
    nn_state_dm.fit(data = meas_result,
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/density_matrix.py", line 354, in fit
    super().fit(
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/neural_state.py", line 614, in fit
    all_grads = self.compute_batch_gradients(k, *batch)
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/neural_state.py", line 439, in compute_batch_gradients
    grad = self.positive_phase_gradients(samples_batch, bases_batch=bases_batch)
  File "/opt/conda/lib/python3.9/site-packages/qucumber/nn_states/neural_state.py", line 382, in positive_

[W 2023-08-25 12:35:21,716] Trial 687 failed with value None.


Trial 687 failed with value None.


KeyboardInterrupt: 

In [ ]:
study.best_params

In [ ]:
# save best params
params["train_info"]["lr"] = study.best_params["lr"]
params["train_info"]["n_gibbs_step"] = study.best_params["k"]

with open('./best_params_setting.yaml', 'w') as yml:
    yaml.dump(params, yml, default_flow_style=False)